![panda](http://res.freestockphotos.biz/thumbs/3/3173-illustration-of-a-giant-panda-eating-bamboo-th.png)

# Pandas Data Cleaning

# Objectives

- Handle missing data, and recognize when different strategies for handling missing data would be appropriate
- Use DataFrame methods (and sometimes lambda functions) to transform data
- Use string methods to transform object-type columns

In [1]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

## To The Center!

That's right - we're still working with Austin Animal Center data! But now, let's check out the Outcomes data, instead of the Intakes data we were working with before.

Data source: https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238

In [6]:
# Read in the animal center outcomes data as df
# Can use parse_dates and pass a column name to read it as a datetime
df = pd.read_csv('../data/Austin_Animal_Center_Outcomes_022822.csv',
                 parse_dates=['DateTime', 'Date of Birth'])

FileNotFoundError: [Errno 2] No such file or directory: '../data/Austin_Animal_Center_Outcomes_022822.csv'

In [ ]:
# Check it out
df.head()

In [ ]:
# Info on the dataframe
df.info()

## Using Questions to Drive Both Exploration and Cleaning

The best way to think about how to both explore and clean your data set is to think about what is interesting in your data, what questions you have, what you want to know and how to actually get there. We're going to ask a few questions of our dataset, and use that to drive our 'cleaning' process (and talk a little bit about how and when to clean data in the process!)

## Question 1: How old are the animals in our dataset?

Let's try to answer this with the `Age upon Outcome` variable:

In [ ]:
df['Age upon Outcome'].value_counts()

Oh no! These aren't numbers! And the data is pretty dirty! Let's see how we can break this column apart, into the Age Number and the Age Unit.

In [ ]:
# Split out the age number and age unit - using string methods!
df['Age Number'] = df['Age upon Outcome'].str.split(" ").str[0]
df['Age Unit'] = df['Age upon Outcome'].str.split(" ").str[1]

In [ ]:
# Sanity check
df.head()

In [ ]:
# Check the data types really quick...
df.info()

In [ ]:
# Need to make our Age Number actual numbers! 
df['Age Number'].astype(int)
# But this throws an error!

Alas! Nulls! Let's check out these null values:

In [ ]:
# Check how many nulls we have in our original Age Upon Outcome column
sum(df['Age upon Outcome'].isna())

### Handling Missing Data

A lot of the times we'll have missing information in our data set. This can sometimes be troublesome in what we're trying to do.

There are a few strategies we can choose from and they each have their pros/cons:

- **Fill with a Relevant Value**
    - If we know what the nulls 'should' be, easy to fill them with that value
    - For numbers, perhaps the null indicates a 0
    - Or, for string columns, might be easier to handle if we fill with "Missing" or "Unknown"
- **Fill with a Reasonable Value**
    - For numeric data, it might be acceptable to fill with a measure of central tendency (mean or median)
    - For categorical/string data, might be acceptable to fill with the most common (mode)
    - But beware! Filling in missing values can lead to you drawing incorrect conclusions. If most of the data from a column are missing, it's going to appear that the value you filled it in with is more common that it actually was!
- **Specify Missing Data**
    - If you plan to fill in missing values, it might make sense to specify that the data was originally missing by creating a new indicator column
    - This can be helpful when you suspect that the fact the data was missing could be important for an analysis.
- **Drop Missing Data**
    - While you should try to keep as much relevant data as possible, sometimes the other methods don't make as much sense and it's better to remove or **drop** the missing data
    - We typically drop missing data if very little data would be lost and/or trying to fill in the values wouldn't make sense for our use case



So: what should we do about the missing Age upon Outcome data?

<br>
<details>
    <summary>Possible Answer</summary>

- It's probably okay to drop the missing data in this case - not very many rows with nulls in that column
- Should look at the percentage of the total (aka what percentage of rows would be dropped) to justify this decision
    
    
Note! There is a Date of Birth column that does not have any nulls! But we're specifically trying to use Age Upon Outcome here because it helps us practice some other tactics for cleaning data.
    
</details>



In [ ]:
# No matter what - better to make this change on a COPY of the dataframe
df_clean = df.copy()

In [ ]:
# Code here to handle nulls
df_clean = df_clean.dropna(subset=['Age upon Outcome'])

In [ ]:
# Sanity check
df_clean.isna().sum()

In [ ]:
# Cool - now we can make our Age Number column an integer
df_clean['Age Number'] = df_clean['Age Number'].astype(int)

In [ ]:
# Sanity check
df_clean.info()

Now let's do something about those Age Values...

### Map, Apply and Applymap

Resource: https://www.geeksforgeeks.org/difference-between-map-applymap-and-apply-methods-in-pandas/

The `.map()` method applies a transformation to every entry in the Series. This transformation  "maps" each value from the Series to a new value. A transformation can be defined by a function, Series, or dictionary.

The `.apply()` method is similar to the `.map()` method for Series, but can only take in functions. 

The `.applymap()` method only works on DataFrames, and applies the same transformation to every element (cell) of that DataFrame.

In [ ]:
# Providing this dictionary to capture age values in # days (not perfect)
age_vals = {
    'years': 365,
    'year': 365,
    'months': 30,
    'month': 30,
    'weeks': 7,
    'week': 7,
    'days': 1,
    'day': 1
}

In [ ]:
# Now, how can we transform the Age Value column to capture these values?
df_clean['Age Unit'] = df_clean['Age Unit'].map(age_vals)

In [ ]:
# Sanity check
df_clean.head()

In [ ]:
# Now, let's make a new Age in Days column!
df_clean['Age in Days'] = (df_clean['Age Number'] * df_clean['Age Unit'])

In [ ]:
# Sanity check
df_clean.head()

In [ ]:
# Now we can visualize!
df_clean['Age in Days'].plot(kind='hist');

In [ ]:
# Of course, we could also try...
# Use .dt.normalize on the DateTime column to access the date!
df_clean['Calculated Age in Days'] = df_clean['DateTime'].dt.normalize() - df_clean['Date of Birth']

In [ ]:
# Check it
df_clean.head()

## Question 2: Are Most of The Animals Already Fixed?

Let's use `.map()` to turn the Sex upon Outcome column into a category with three values: Fixed, Intact, or Unknown. 

In [ ]:
# Check out what's currently in that column
df_clean['Sex upon Outcome'].unique()

In [ ]:
# We can write a function for this
def fixed_mapper(status):
    '''
    Takes in the current status of animals and outputs whether they have been fixed
    '''
    if status in ['Neutered Male', 'Spayed Female']:
        return 'Fixed'
    elif status in ['Intact Male', 'Intact Female']:
        return 'Intact'
    else:
        return 'Unknown'

In [ ]:
# Now let's use it!
df_clean['Grouped Sex upon Outcome'] = df_clean['Sex upon Outcome'].map(fixed_mapper)
df_clean[['Sex upon Outcome', 'Grouped Sex upon Outcome']].head()

In [ ]:
# We had a null in that column before, did that change?
df_clean.isna().sum()

In [ ]:
# Visualize it!
df_clean['Grouped Sex upon Outcome'].value_counts().plot(kind='pie');

In [ ]:
df_clean['Grouped Sex upon Outcome'].value_counts().plot(kind='bar');

## A Quick Aside - for Lambda Functions!

Simple functions can be defined just when you need them, when you would call the function. These are called **lambda functions**. These functions are **anonymous** and disappear immediately after use.

These can often be great as map transformation functions, but they can only do so much.

Let's use them to fill some nulls!

In [ ]:
# Check which columns still have null values
df_clean.isna().sum()

Let's look at Outcome Subtype - a lot of nulls, why do we think that is? What would be an appropriate strategy here?

<br>
<details>
    <summary>Possible Answer</summary>

- Might be that the Type doesn't have Subtypes, or that there hasn't been an outcome yet
- Would be appropriate to fill with N/A
    
</details>





In [ ]:
# Let's try it!
# Note: need to use x is np.nan as our if condition
df_clean['Outcome Subtype'].map(lambda x: "N/A" if x is np.nan else x)

In [ ]:
# BUT! Pandas has a function for this
df_clean['Outcome Subtype'] = df_clean['Outcome Subtype'].fillna("N/A")
df_clean['Outcome Subtype']

In [ ]:
# We can use one of these methods to fill the nulls for Outcome Type too
# Explore the column here
df_clean['Outcome Type'].unique()

In [ ]:
# Fill nulls here
df_clean['Outcome Type'] = df_clean['Outcome Type'].fillna("Unknown")

In [ ]:
# Sanity Check
df_clean.isna().sum()

### One More Strategy: Build a Missing Indicator

While this doesn't directly answer a question we have, let's go ahead and build a new column to indicate where Name is null (just to demo).

This can be quite useful when modeling, if the data being missing actually means something!

In [ ]:
# Quick pandas method for this
df_clean['Name Missing'] = df_clean['Name'].isna()
df_clean.head()

Might be more interesting to build an indicator of where "Mix" is in the Breed column

In [ ]:
# Let's do it
df_clean['Mix Indicator'] = df_clean['Breed'].str.contains(" Mix")

In [ ]:
# Sanity check
df_clean.head()

## Comparing Before and After

We can now see all the work we did!

In [ ]:
# Original data
df.info()

In [ ]:
# Missing data cleaned
df_clean.info()

# Level Up: `.applymap()`

`.applymap()` is used to apply a transformation to each element of a DataFrame.

In [ ]:
# This line will apply the base `type()` function to 
# all entries of the DataFrame.

df.applymap(type).head()

# Level Up: Faster NumPy Methods

In general, `np.where()` and `np.select()` are faster than `map()`. This won't matter too much with reasonably-sized data but can be a consideration for ***big data***.

In [ ]:
# Let's copy the dataframe to play around with
level_up = df_clean.copy()

In [ ]:
level_up.head()

## NumPy's `where()` Method

In [ ]:
# First pass a condition
level_up['Adopted1'] = np.where(level_up['Outcome Type'] == 'Adoption',
                                True,  # What to fill with if the condition is true
                                False)  # What to fill with if not true
level_up.head()

In [ ]:
level_up['Adopted2'] = level_up['Outcome Type'].map(lambda x: True if x == 'Adoption' else False)
level_up.head()

In [ ]:
# Let's time how long it takes np.where() to run by running it multiple times
%timeit np.where(level_up['Outcome Type'] == 'Adoption', True, False)

In [ ]:
# Let's time how long it takes .map() to run by running it multiple times
%timeit level_up['Outcome Type'].map(lambda x: True if x == 'Adoption' else False)

## NumPy's `select()` Method

Again, `numpy` will be faster:

In [ ]:
# Define our list of conditions
conditions = [level_up['Sex upon Outcome'] == 'Neutered Male',
              level_up['Sex upon Outcome'] == 'Spayed Female',
              level_up['Sex upon Outcome'] == 'Intact Male',
              level_up['Sex upon Outcome'] == 'Intact Female',
              level_up['Sex upon Outcome'] == 'Unknown',
              level_up['Sex upon Outcome'].isna()]

# Define a matching list of outcomes
choices = ['Fixed', 'Fixed', 'Intact', 'Intact', 'Unknown', 'Unknown']

In [ ]:
# Use np.select
level_up['Grouped Sex upon Outcome1'] = np.select(conditions, choices)
level_up['Grouped Sex upon Outcome1']

In [ ]:
# Check we got the same results with np.where()
(level_up['Grouped Sex upon Outcome1'] != level_up['Grouped Sex upon Outcome']).sum()

In [ ]:
# Let's time how long it takes .map() to run by running it multiple times
%timeit level_up['Sex upon Outcome'].map(fixed_mapper)

In [ ]:
# Let's time how long it takes np.select() to run by running it multiple times
%timeit np.select(conditions, choices)